In [1]:
import pongGym
import random
import numpy as np
import os
env = pongGym.DoublePong()
PAD_HEIGHT = 80
HALF_PAD_HEIGHT = PAD_HEIGHT // 2

pygame 2.1.0 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def encode(state):
    pd1y = state[21]
    pd2y = state[23]
    st = [(max(0, state[i*4+0] + state[i*4+2] * 3), abs(state[i*4+1] + state[i*4+3] * 3), state[i*4+2], state[i*4+3]) for i in range(5)]
    st.sort(key = lambda s: s[0])

    now = 1
    re = 0
#     print(st)
    for i in range(5):
        ty = 0
        if(st[i][1] > pd1y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd1y - HALF_PAD_HEIGHT):
            ty = 2
        re += now * ty
        now *= 3
        ty = 0
        if(st[i][1] > pd2y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd2y - HALF_PAD_HEIGHT):
            ty = 2
        re += now * ty
        now *= 3
    return re
state = env.reset()

In [3]:

q_table = np.zeros([(3*3)**5, env.action_space.n])
weight_path = "q_learning_{}.npy".format("exp3")
if os.path.isfile(weight_path):
    with open(weight_path, 'rb') as f:
        q_table = np.load(f)

In [4]:
print(q_table)

[[0.49518548 0.42708754 0.42670723 0.42130743]
 [0.07826366 0.         0.         0.        ]
 [0.44867809 0.17699174 0.15525848 0.10870351]
 ...
 [0.88760417 0.92424125 0.98459873 1.01769356]
 [0.68426506 0.60277838 0.65810858 0.88501424]
 [0.32775011 0.39207796 0.340882   0.37915287]]


In [7]:
import random
import math
from IPython.display import clear_output

# Hyperparameters
Init_alpha = 0.08
gamma = 0.9236
Init_epsilon = 0.2

# For plotting metrics
all_epochs = []
all_penalties = []
all_reward = 0

for i in range(1, 100001):
    alpha = math.exp(-0.001 * i) * Init_alpha
    epsilon = math.exp(-0.0001 * i) * Init_epsilon

    state = env.reset()
    state = encode(state)
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        next_state, reward, done, info = env.step(action)
        next_state = encode(next_state)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1
        all_reward += reward
        state = next_state
        epochs += 1
        
    if i % 100 == 0:
#         clear_output(wait=True)
        print(f"Episode: {i}")
        print(f"average reward: {all_reward / 100}")
        all_reward = 0

print("Training finished.\n")

Episode: 100
average reward: 697.8
Episode: 200
average reward: 511.61
Episode: 300
average reward: 554.11
Episode: 400
average reward: 563.12
Episode: 500
average reward: 660.24
Episode: 600
average reward: 743.9
Episode: 700
average reward: 825.75
Episode: 800
average reward: 800.87
Episode: 900
average reward: 873.36
Episode: 1000
average reward: 975.85
Episode: 1100
average reward: 1056.06
Episode: 1200
average reward: 1049.88
Episode: 1300
average reward: 1099.52
Episode: 1400
average reward: 1192.26
Episode: 1500
average reward: 1226.77
Episode: 1600
average reward: 1264.89
Episode: 1700
average reward: 1308.48
Episode: 1800
average reward: 1343.52
Episode: 1900
average reward: 1440.62
Episode: 2000
average reward: 1468.75
Episode: 2100
average reward: 1522.8
Episode: 2200
average reward: 1587.21
Episode: 2300
average reward: 1485.65
Episode: 2400
average reward: 1569.57
Episode: 2500
average reward: 1562.09
Episode: 2600
average reward: 1663.28
Episode: 2700
average reward: 1664

Episode: 21500
average reward: 2225.51
Episode: 21600
average reward: 2250.94
Episode: 21700
average reward: 2248.22
Episode: 21800
average reward: 2204.53
Episode: 21900
average reward: 2238.23
Episode: 22000
average reward: 2256.53
Episode: 22100
average reward: 2243.28
Episode: 22200
average reward: 2273.57
Episode: 22300
average reward: 2225.45
Episode: 22400
average reward: 2297.2
Episode: 22500
average reward: 2221.4
Episode: 22600
average reward: 2275.15
Episode: 22700
average reward: 2239.19
Episode: 22800
average reward: 2295.12
Episode: 22900
average reward: 2261.33
Episode: 23000
average reward: 2221.0
Episode: 23100
average reward: 2244.91
Episode: 23200
average reward: 2212.95
Episode: 23300
average reward: 2292.63
Episode: 23400
average reward: 2269.1
Episode: 23500
average reward: 2239.71
Episode: 23600
average reward: 2251.11
Episode: 23700
average reward: 2241.52
Episode: 23800
average reward: 2200.99
Episode: 23900
average reward: 2196.94
Episode: 24000
average reward

Episode: 42600
average reward: 2326.9
Episode: 42700
average reward: 2216.43
Episode: 42800
average reward: 2268.84
Episode: 42900
average reward: 2246.66
Episode: 43000
average reward: 2332.31
Episode: 43100
average reward: 2296.71
Episode: 43200
average reward: 2311.91
Episode: 43300
average reward: 2248.41
Episode: 43400
average reward: 2272.04
Episode: 43500
average reward: 2244.06
Episode: 43600
average reward: 2255.7
Episode: 43700
average reward: 2300.14
Episode: 43800
average reward: 2246.67
Episode: 43900
average reward: 2312.36
Episode: 44000
average reward: 2243.88
Episode: 44100
average reward: 2276.99
Episode: 44200
average reward: 2290.1
Episode: 44300
average reward: 2231.1
Episode: 44400
average reward: 2283.06
Episode: 44500
average reward: 2297.42
Episode: 44600
average reward: 2276.85
Episode: 44700
average reward: 2320.57
Episode: 44800
average reward: 2320.44
Episode: 44900
average reward: 2286.7
Episode: 45000
average reward: 2267.77
Episode: 45100
average reward:

Episode: 63700
average reward: 2314.47
Episode: 63800
average reward: 2290.91
Episode: 63900
average reward: 2264.91
Episode: 64000
average reward: 2298.99
Episode: 64100
average reward: 2335.84
Episode: 64200
average reward: 2297.56
Episode: 64300
average reward: 2312.13
Episode: 64400
average reward: 2272.51
Episode: 64500
average reward: 2306.54
Episode: 64600
average reward: 2300.31
Episode: 64700
average reward: 2313.48
Episode: 64800
average reward: 2265.09
Episode: 64900
average reward: 2307.72
Episode: 65000
average reward: 2306.33
Episode: 65100
average reward: 2284.48
Episode: 65200
average reward: 2280.29
Episode: 65300
average reward: 2300.8
Episode: 65400
average reward: 2340.81
Episode: 65500
average reward: 2279.02
Episode: 65600
average reward: 2265.27
Episode: 65700
average reward: 2311.61
Episode: 65800
average reward: 2301.52
Episode: 65900
average reward: 2315.95
Episode: 66000
average reward: 2280.25
Episode: 66100
average reward: 2244.43
Episode: 66200
average rew

Episode: 84800
average reward: 2302.46
Episode: 84900
average reward: 2314.16
Episode: 85000
average reward: 2306.0
Episode: 85100
average reward: 2285.17
Episode: 85200
average reward: 2301.66
Episode: 85300
average reward: 2320.12
Episode: 85400
average reward: 2306.34
Episode: 85500
average reward: 2335.23
Episode: 85600
average reward: 2317.39
Episode: 85700
average reward: 2281.07
Episode: 85800
average reward: 2324.23
Episode: 85900
average reward: 2354.21
Episode: 86000
average reward: 2279.67
Episode: 86100
average reward: 2325.52
Episode: 86200
average reward: 2300.87
Episode: 86300
average reward: 2293.83
Episode: 86400
average reward: 2294.1
Episode: 86500
average reward: 2258.51
Episode: 86600
average reward: 2258.85
Episode: 86700
average reward: 2301.3
Episode: 86800
average reward: 2326.67
Episode: 86900
average reward: 2249.77
Episode: 87000
average reward: 2314.55
Episode: 87100
average reward: 2289.66
Episode: 87200
average reward: 2301.93
Episode: 87300
average rewar

In [9]:
with open(weight_path, 'wb') as f:
    np.save(f, q_table)
    

In [6]:
import time
total_epochs, total_penalties = 0, 0
episodes = 10
total_reward = 0
for _ in range(episodes):
    state = env.reset()
    state = encode(state)
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        state = encode(state)
        if reward == -10:
            penalties += 1
        
        epochs += 1
        total_reward += reward
#         env.render()
#         time.sleep(1/60)
    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Average reward per episode: {total_reward / episodes}")

Results after 10 episodes:
Average timesteps per episode: 10801.0
Average penalties per episode: 52.8
Average reward per episode: 2118.2


In [6]:
with open("weight.coe", 'w') as f:
    f.write("memory_initialization_radix=10;\n")
    f.write("memory_initialization_vector=\n")
    for i in range(3**10):
        f.write(str(np.argmax(q_table[i])))
        if(i == 3**10 - 1):
            f.write(";\n")
        else:
            f.write(",\n")